In [1]:
import pandas as pd
obj=pd.read_csv('data_clean2_09_12_2018.csv')
len(obj)

75056

In [2]:
material_ids_after_clean=[]
for i in range(len(obj["material_id"])):
    material_ids_after_clean.append(obj["material_id"][i])

In [3]:
material_ids_after_clean[1:3]

['mp-10000', 'mp-10003']

In [4]:
obj.head()

,material_id,pretty_formula
0,mp-1000,BaTe
1,mp-10000,Hf2S
2,mp-10003,Nb4CoSi
3,mp-10004,Mo3P
4,mp-1000440,CrN


In [5]:
#check all the elements of the material_ids_after_clean
#store the data's point group, space group and formula

In [6]:
#get all the data in the cleaned dataset

from pymatgen import MPRester                                                     
import urllib.request                                                             
import json

                                                                                  
if __name__ == "__main__":                                                        
    MAPI_KEY = "zQIrujyVEwEhTBRW"  # You must change this to your Materials API key! (or set MAPI_KEY env variable)
                                                                                  
    # fetch list of a list of all available materials                             
    #with urllib.request.urlopen('https://www.materialsproject.org/rest/v1/materials//mids') as myurl:
        #data = json.loads(myurl.read().decode())                                  
        #material_ids = data['response'] # 75,000'ish material IDs are returned 
                                                                                  
                                                                                  
    with MPRester(MAPI_KEY) as m: # object for connecting to MP Rest interface 
        criteria={'material_id': {'$in':material_ids_after_clean[:]}} # to avoid straining the servers, this is only using the first 4 materials
        #properties=['energy', 'pretty_formula','total_electrons','density']
        properties=['material_id','pretty_formula','energy','spacegroup'] 
        # list a few quanteties of interest
        data = m.query(criteria, properties)                                      
        #print(data[:10])

[{'material_id': 'mp-1000', 'pretty_formula': 'BaTe', 'energy': -8.64734519, 'spacegroup': {'source': 'spglib', 'symbol': 'Fm-3m', 'number': 225, 'point_group': 'm-3m', 'crystal_system': 'cubic', 'hall': '-F 4 2 3'}}, {'material_id': 'mp-10000', 'pretty_formula': 'Hf2S', 'energy': -54.26524496, 'spacegroup': {'source': 'spglib', 'symbol': 'P6_3/mmc', 'number': 194, 'point_group': '6/mmm', 'crystal_system': 'hexagonal', 'hall': '-P 6c 2c'}}, {'material_id': 'mp-10003', 'pretty_formula': 'Nb4CoSi', 'energy': -110.0939723, 'spacegroup': {'source': 'spglib', 'symbol': 'P4/mcc', 'number': 124, 'point_group': '4/mmm', 'crystal_system': 'tetragonal', 'hall': '-P 4 2c'}}, {'material_id': 'mp-10004', 'pretty_formula': 'Mo3P', 'energy': -157.39830964, 'spacegroup': {'source': 'spglib', 'symbol': 'I-42m', 'number': 121, 'point_group': '-42m', 'crystal_system': 'tetragonal', 'hall': 'I -4 2'}}, {'material_id': 'mp-1000440', 'pretty_formula': 'CrN', 'energy': -18.64352006, 'spacegroup': {'source': 

In [9]:
data[6]

{'material_id': 'mp-1000588',
 'pretty_formula': 'U2IrC2',
 'energy': -51.54211016,
 'spacegroup': {'source': 'spglib',
  'symbol': 'I4/mmm',
  'number': 139,
  'point_group': '4/mmm',
  'crystal_system': 'tetragonal',
  'hall': '-I 4 2'}}

In [121]:
#the required elpasolite structure has space group number 225, symbol as Fm-3m
from pymatgen import Composition, Element

needed_list=[]
for k in range(len(data)):
    compound=Composition(data[k]["pretty_formula"])
        #fit the condition that # of elements=4 and space group number is 225
    #if len(compound)==4 and data[i]["spacegroup"]["number"]==225:
    if len(compound)==4:
        #fit the formula is ABC2D6
        #build compustition_permutation_list
        compustition_permutation_list=[]
        
        ele_list=[]
        for i in compound:
            ele_list.append(str(i))
        elements_list_permutation_list=[list(p) for p in permutations(ele_list)]
        elpasolite_number=["","","2","6"]
        for elements_list_permutation in elements_list_permutation_list:
            #for a elements list, build sparse elements+number string
            ele_list=[]
            j=0
            for element in elements_list_permutation:
                ele_list.append(str(element)+elpasolite_number[j])
                j+=1
            #build permuations of sparse elements+number string
            elements_split_list=[list(p) for p in permutations(ele_list)]

            #build permuations of condense elements+number string
            elements_conca_list=[]
            for elements_split in elements_split_list:
                elements_list=""
                for elements in elements_split:
                    elements_list+=elements
                elements_conca_list.append(elements_list)

            #add condensed permuations of string to the whole
            compustition_permutation_list+=elements_conca_list
        #if data[k]["pretty_formula"] in compustition_permutation_list:
            #needed_list.append(k)
        
        for j in compustition_permutation_list:
            if data[k]["pretty_formula"]==j:
                needed_list.append(k)
                break

In [95]:
type(data[1]["pretty_formula"])

str

In [138]:
len(needed_list)

1114

In [139]:
set_list=[]
a=set()
for i in needed_list:
    len_before=len(a)
    a.add(data[i]["pretty_formula"])
    len_after=len(a)
    if len_after>len_before:
        set_list.append(i)
    #print(i,data[i]["pretty_formula"])


In [141]:
len(set_list)

848

In [142]:
data_elpasolite=[]
for i in set_list:
    data_elpasolite.append(data[i])

In [144]:
data_elpasolite[:3]

[{'material_id': 'mp-10093',
  'pretty_formula': 'Sr2YbNbO6',
  'energy': -144.71117075,
  'spacegroup': {'source': 'spglib',
   'symbol': 'P2_1/c',
   'number': 14,
   'point_group': '2/m',
   'crystal_system': 'monoclinic',
   'hall': '-P 2ybc'}},
 {'material_id': 'mp-1019568',
  'pretty_formula': 'CaAl2SiO6',
  'energy': -298.39845493,
  'spacegroup': {'source': 'spglib',
   'symbol': 'P1',
   'number': 1,
   'point_group': '1',
   'crystal_system': 'triclinic',
   'hall': 'P 1'}},
 {'material_id': 'mp-1019719',
  'pretty_formula': 'CsSi2BO6',
  'energy': -597.32726776,
  'spacegroup': {'source': 'spglib',
   'symbol': 'I4_1/acd',
   'number': 142,
   'point_group': '4/mmm',
   'crystal_system': 'tetragonal',
   'hall': 'I 4bw 2aw -1bw'}}]